In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

knn = NearestNeighbors()
td_idf_vectorizer = TfidfVectorizer()

In [2]:
listings_df_pd = pd.read_csv("listings.csv")
# listings_df_pd.to_parquet("listing.pkt")
# listings = spark.read.parquet("listing.pkt")
# listings_not_null = listings[listings.name.isNotNull()]

In [3]:
listings_df_pd_filtered = listings_df_pd[~listings_df_pd.name.isnull()].reset_index(drop=True)

In [4]:
train, test = train_test_split(listings_df_pd_filtered, test_size=0.2, random_state=42)

In [5]:
it_idf_names = td_idf_vectorizer.fit_transform(test.name)

In [6]:
knn.fit(X=it_idf_names)

NearestNeighbors()

In [7]:
five_closest = [list(knn.kneighbors(d,6)[1][0][1:] )for d in it_idf_names]

In [8]:
test['ground_truth'] = [test.iloc[d].id.to_list() for d in five_closest ]

<ipython-input-8-3e51ce16ee12>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['ground_truth'] = [test.iloc[d].id.to_list() for d in five_closest ]


In [9]:
test.to_parquet("train_test_data/test.pkt")
train.to_parquet("train_test_data/train.pkt")

In [10]:
t = test.iloc[55].ground_truth

In [11]:
test.iloc[55]['name']

'Cozy house with terrace in the centre of Barcelona'

In [12]:
test.loc[test.id.isin(t)].name

18853                      COZY house with TERRACE
3335               Cozy Room in the centre of BCN!
4391        Cozy apartment in the centre of Gracia
9150                            House with terrace
14999    Beautiful House in the heart of Barcelona
Name: name, dtype: object